In [2]:
from azure.identity import AzureCliCredential
from agent_framework import ChatAgent
from agent_framework.azure import AzureOpenAIChatClient

agent = ChatAgent(
    chat_client=AzureOpenAIChatClient(
        credential=AzureCliCredential()
    ),
    name="Assistant",
    instructions="You are a helpful assistant."
)

thread = agent.get_new_thread()

In [3]:
# Run the agent and append the exchange to the thread
response = await agent.run("Tell me a short pirate joke.", thread=thread)
print(response.text)

Sure! Here's a joke for you:

Why don't pirates take a shower before they walk the plank?  
Because they just wash up on shore! 🏴‍☠️


In [ ]:
def to_serializable(obj):
    if hasattr(obj, "to_dict"):
        return obj.to_dict()
    elif hasattr(obj, "__dict__"):
        return obj.__dict__
    else:
        return str(obj)

import json
import tempfile
import os

# Serialize the thread state
serialized_thread = await thread.serialize()

serialized_json = json.dumps(serialized_thread, default=to_serializable)

# Example: save to a local file (replace with DB or blob storage in production)
temp_dir = tempfile.gettempdir()
file_path = os.path.join(temp_dir, "agent_thread.json")
with open(file_path, "w") as f:
    f.write(serialized_json)

{'service_thread_id': None, 'chat_message_store_state': {'messages': [<agent_framework._types.ChatMessage object at 0x000001AE40C9F770>, <agent_framework._types.ChatMessage object at 0x000001AE40DAAD70>]}}


In [21]:
# Read persisted JSON
with open(file_path, "r") as f:
    loaded_json = f.read()

reloaded_data = json.loads(loaded_json)

from agent_framework import ChatMessage

# Convert dicts to ChatMessage objects
for i, msg in enumerate(reloaded_data["chat_message_store_state"]["messages"]):
    if isinstance(msg, dict):
        reloaded_data["chat_message_store_state"]["messages"][i] = ChatMessage.from_dict(msg)


# Deserialize the thread into an AgentThread tied to the same agent type
resumed_thread = await agent.deserialize_thread(reloaded_data)

In [ ]:
# Continue the conversation with resumed thread
response = await agent.run("Now tell that joke in the voice of a pirate.", thread=resumed_thread)
print(response.text)

Arrr, ye be askin' fer some pirate flair, eh? Alright, here goes:

"Why don’t we scurvy sea dogs take a bath afore walkin’ the plank? Arrr... ’cause we just wash up on shore, matey! Har har har!" 🏴‍☠️


: 